# pinkfish-challenge

Buy on the close on the SAME day a new 20 day high is set

In [1]:
import datetime
import pandas as pd
import pinkfish as pf

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2016, 1, 1)
end = datetime.datetime.now()

Define high trade periods

In [3]:
period = 20

Timeseries

In [4]:
# Fetch timeseries
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add technical indicator: X day high, and X day low
period_high = pd.Series(ts.close).rolling(period).max()
ts['period_high'] = period_high

# Finalize timeseries
ts, start = pf.finalize_timeseries(ts, start, dropna=True)

# Create tlog and dbal objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

Algorithm

In [5]:
pf.TradeLog.cash = capital

for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    high = row.high; low = row.low; close = row.close; 
    end_flag = pf.is_last_row(ts, i)
    shares = 0

    # Buy
    if tlog.shares == 0:
        if high > row.period_high:
            shares = tlog.buy(date, close)
    # Bell
    else:
        if end_flag:
            shares = tlog.sell(date, close)

    if shares > 0:
        print("{0} BUY  {1} {2} @ {3:.2f}".format(
              date, shares, symbol, close))
    elif shares < 0:
        print("{0} SELL {1} {2} @ {3:.2f}".format(
              date, -shares, symbol, close))

    # Record daily balance
    dbal.append(date, close, high, low)

2016-02-22 00:00:00 BUY  51 SPY @ 194.78
2024-12-27 00:00:00 SELL 51 SPY @ 595.01


In [6]:
tlog = tlog.get_log()
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,2016-02-22,194.78,2024-12-27,595.01,400.23,20411.73,51,20411.73,LONG,SPY


The first 20 day high occurred on 2/22/2016, so we bought on the close that day.
We held onto the present, then sold the positon.